#Les données

In [ ]:
file = open('Ens_A.txt', 'rt', encoding='utf-8')
text = file.read()
A_data = text.split("\n")
help=A_data[0].replace('\ufeff','')
del A_data[0]
A_data.insert(0,help)
A=[]
for q in A_data:
  a=q.split(' ')
  A.append(' '.join(a))
A_data=A
    

#Question
file = open('Ens_Q.txt', 'rt', encoding='utf-8')
text = file.read()
Q_data = text.split("\n")
help=Q_data[0].replace('\ufeff','')
del Q_data[0]
Q_data.insert(0,help)





for a in range(len(A_data)):
    A_data[a] = '<start> '+A_data[a]+' <end>'
    
for a in range(len(Q_data)):
    Q_data[a] = '<start> '+Q_data[a]+' <end>'

print('la taille des Questions : {}'.format(len(Q_data)))
print('la taille des Réponses  : {}'.format(len(A_data)))

la taille des Questions : 820
la taille des Réponses  : 820


# Prétraitement

In [ ]:
def tokenize(x):
    tokenizer = Tokenizer(char_level=False,filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
    tokenizer.fit_on_texts(x)
    
    return tokenizer.texts_to_sequences(x), tokenizer

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
        print('Length Max: {}'.format(length))
        print()
    return pad_sequences(x, maxlen=length, padding='post')

def preprocess(Q,A):
 
    preprocess_Q, Q_tk = tokenize(Q)
    preprocess_A, A_tk = tokenize(A)

    preprocess_Q = pad(preprocess_Q)
    preprocess_A = pad(preprocess_A)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_A = preprocess_A.reshape(*preprocess_A.shape, 1)

    return preprocess_Q, preprocess_A, Q_tk, A_tk




preproc_Q, preproc_A, Q_tokenizer, A_tokenizer = preprocess(Q_data, A_data)

print(Q_tokenizer.word_index) #Dict Q
print(A_tokenizer.word_index) #Dict A


max_Q_sequence_length = preproc_Q.shape[1]
max_A_sequence_length = preproc_A.shape[1]

Q_vocab_size = len(Q_tokenizer.word_index)+1
A_vocab_size = len(A_tokenizer.word_index)+1

print('\n')

print('Data Preprocessed')
print("Max Q sentence length:", max_Q_sequence_length)
print("Max A sentence length:", max_A_sequence_length)
print("Q vocabulary size:", Q_vocab_size)
print("A vocabulary size:", A_vocab_size)

Length Max: 24

Length Max: 25

{'<start>': 1, '<end>': 2, 'في': 3, 'العلوم': 4, 'ما': 5, 'و': 6, 'سلك': 7, 'لسلك': 8, 'التقنيات': 9, 'عدد': 10, 'هل': 11, 'من': 12, 'الهندسة': 13, 'بمدينة': 14, 'بكلية': 15, 'توجد': 16, 'المدرسة': 17, 'للتكنولوجيا': 18, 'هي': 19, 'المتوفرة': 20, 'المتاحة': 21, 'الماستر': 22, 'كلية': 23, 'جدع': 24, 'المسالك': 25, 'والتقنيات': 26, 'المقاعد': 27, 'دبلوم': 28, 'متى': 29, 'بنسبة': 30, 'الوطنية': 31, 'الجامعي': 32, 'الاجازة': 33, 'الدبلوم': 34, 'كمياء': 35, 'العليا': 36, 'بعد': 37, 'المشترك': 38, 'اعلاميات': 39, 'فيزياء': 40, 'رياضيات': 41, 'على': 42, 'التجارة': 43, 'الإجازة': 44, 'إجتياز': 45, 'للتسير': 46, 'الجذع': 47, 'السنتين': 48, 'الطالب': 49, 'الوحدة': 50, 'بسلك': 51, 'كيف': 52, 'مسلك': 53, 'التحضريتين': 54, 'جيولوجيا': 55, 'بيولوجيا': 56, 'المقاعدة': 57, 'لكلية': 58, 'والتقنات': 59, 'يتم': 60, 'السنة': 61, 'ماهي': 62, 'فصل': 63, 'الدكتوره': 64, 'ماذا': 65, 'هو': 66, 'كم': 67, 'ملال': 68, 'مدة': 69, 'لسنة': 70, 'طنجة': 71, 'التسجيل': 72, 'النقطة': 73, 

#Utilisation des modèles sauvegarder

In [ ]:
encoder_model = load_model('Ens_Encoder_model.h1')

c:\users\ucefb\appdata\local\programs\python\python38\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [ ]:
decoder_model = load_model('Ens_Decoder_model.h1')

In [ ]:
encoder_model.summary()
decoder_model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 24)                0         
_________________________________________________________________
enc_embedding1 (Embedding)   (None, 24, 256)           167424    
_________________________________________________________________
lstm_5 (LSTM)                [(None, 256), (None, 256) 525312    
Total params: 692,736
Trainable params: 692,736
Non-trainable params: 0
_________________________________________________________________
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, None)         0                                            
_____________________________________________________________________

Prédicter les réponses

toPredictible : tranforme les questions en vecteurs


get answer : prédicter la réponse 


In [ ]:
def toPredectible(qes, tok, shape):
    tt = []
    for q in qes.split(' '):
        if q in tok.word_index:
            tt.append(tok.word_index[q])
        else:
            tt.append(0)
    return pad([tt], shape)
def get_answer(query):
    query = '<start> '+query+' <end>'
    query = toPredectible(query,Q_tokenizer,24)
#     predicteble = np.zeros((1, 24),dtype='float32')
#     predicteble[0] = query
    states_value = encoder_model.predict(query)
    
    a,b,c = decoder_model.predict( [preproc_A[-1][0]] + states_value)
    i = np.argmax(a[0][0])
    answer = A_tokenizer.index_word[i]
    while True:
        a,b,c = decoder_model.predict( [np.array([i])] + [b,c])
        i = np.argmax(a[0][0])
        if(i==2):
            break
        answer =answer+' '+ A_tokenizer.index_word[i]
    return answer

In [ ]:
get_answer('ما شروط التسجيل في سلك الاجازة و الاجازة المهنية')

'ان يكون الطلب حاصل على شهادة البكالوريا او شهادة البكالوريا المهنية'